# An interactive notebook with the FRB/US model
This notebook is based on the the FRB/US model packages located here https://www.federalreserve.gov/econres/us-models-package.htm. The packages provides an Eviews https://www.eviews.com/ implementation. The Eviews modelspecification has been transformed to ModelFlow BusinessLogic specification. 

You can experiment with the model using a simple interactive interface. 

FRB/US comes in two flawors: **VAR** in which expectaions are generated by backward looking equations, and **MCE** in which expectation are based on leaded values. 
This notebook implement the **VAR** flavour.

The tool used is highly experimental and the results are purely for illustration. 

You can run each cell by pressing the run tool or shift+enter
To run all cells use select cell/run all in the menu bar. 

# Import Python libraries

In [ ]:
%matplotlib inline

from modelclass import model
import modelmf
model.modelflow_auto()

# Load the model and run a  baseline

In [ ]:
mfrbus, basedf =  model.modelload('mfrbusvar.json')
res = mfrbus(basedf,'2020q1','2025q4',silent=1,stats=0)

# Define an interactive input widget
This widget allows you to input 

In [ ]:
slidedef = {'Initial taylor':                {'var':'RFFINTAY_AERR','value':0.0,'min':-3, 'max':3,'op':'+'},
            'Initial taylor inital impulse': {'var':'RFFINTAY_AERR','value':0.0,'min':-3, 'max':3,'op':'+impulse'},
            'Minimum federal funds':         {'var':'RFFMIN',       'value':0.0,'min':-20,'max':5,'op':'='       ,'step':1.0}
            }
radiodef = {'Monetary policy reaction':
            [['Inertial Taylor rule','DMPINTAY'],
             ['Taylor rule','DMPTAY'],
             ['Taylor rule with unemployment gap','DMPTLR'],
             ['Estimated rule','DMPALT'],
             ['Generalized rule','DMPGEN'],
             ['Exogenous nominal funds rate', 'DMPEX'],
             ['Exogenous real funds rate', 'DMPRR']
            ]
           }
           
checkdef = [('Timing of zero lower bound', 'DMPTRSH' ,True) ]

input = mfrbus.inputwidget('2020q1','2025q4',slidedef=slidedef,checkdef=checkdef,radiodef=radiodef
                   ,showout=True,varpat='RFF XGDPN RFFMIN GFSRPN DMPTRSH',trans=mfrbus.var_description)

# Try effect of different policy assumptions 
We try to decrease interest rate by 2 percent and look at the models assumptions 
regarding monetary policy rule. 

After the experiments are ru, the results are displayed in a widget which allows the user to 
select variables and different options - thy it out 

In [ ]:
# Define the experiment
experiments = [
  ['Inertial Taylor rule', 'DMPINTAY'],
  ['Taylor rule', 'DMPTAY'],
  ['Taylor rule with unemployment gap', 'DMPTLR'],
  ['Estimated rule', 'DMPALT'],
  ['Generalized rule', 'DMPGEN'],
  ['Exogenous nominal funds rate', 'DMPEX'],
  ['Exogenous real funds rate', 'DMPRR']]
newbasedf = basedf.copy()                     # make a new dataframe
mfrbus.basedf = newbasedf 
newbasedf.loc[:,[v for e,v in experiments]]=0 # Set all policy dummies to 0

for experiment,var in experiments:            # loop over the policy options and update 
    altdf=newbasedf.mfcalc(f'''\
          rffintay_aerr = rffrule_aerr - 2       
          rffmin        = 0.0        
          {var} = 1
    ''')
    result = mfrbus(altdf,'2020q1','2025q4',keep=f'Monetary policy rule:{experiment}')

mfrbus.keep_viz('rff xgdp rg10p',legend = True)

# The input to equations can be traced back. 
and also the propagation of results can be traced.

In [ ]:
with mfrbus.set_smpl('2020q1','2021q2'):
    mfrbus.rffgen.draw(up=1,down=0,HR=0,last=False,svg=1,pdf=1)

In [ ]:
mfrbus.rffrule.frml